# **Project: Classically verifiable quantum advantage from a computational Bell test**

**Developed and authored by QUIP team members:** Filippo Tramonto, Ban Tran, Nahid Binandeh Dehaghani



## **Implementation of Step 1**
The first step involves implementing a toy version of the chosen algorithm, which simplifies the problem addressed in the original paper. We conducted a scalable implementation, ensuring it was thoroughly checked and tested.
To this end, we covered the following steps:
*   Problem Encoding
*   Key algorithmic components implementation
*   Readout and post-processing

## **Key implementation aspects and the methodologies**

* **Quantum algorithm**

Our focus in this step is to implement the quantum algorithm, key component of the protocol discussed in the paper, i.e. the quantum algorithm that implements the computation of Trapdoor Claw-Free Function (TCF) of Rabin, $ f_N(x) =  x^2 \!\!\! \mod \!\! N $, which will be explained in details in the next sections.

The quantum algorithm involves preparing the state $ |\psi\rangle = \sum_x |x\rangle_x |x^2 \!\!\! \mod \!\! N\rangle_y $, where the subscripts $ x $ and $ y $ indicate the quantum registers $ x $ and $ y $.

* **Quantum Circuits**

The paper propose four different quantum circuit designs for implementing the encoding of the TCF, $f_N(x) = x^2 \!\!\! \mod \!\! N N$. It describes also a experimental implementation on quantum hardware of neutral atoms coupled with Ridberg states

1. **Classical algorithms Converted to Quantum**
* **Karatsuba Multiplication Circuit.** This circuit is based on the classical Karatsuba algorithm for efficient integer multiplication. It is adapted for quantum computation by making it reversible and then optimized using a scheme to discard unnecessary "garbage" bits.
* **Schoolbook Multiplication Circuit.** Similar to the Karatsuba circuit, this classical algorithm for integer multiplication is made reversible for quantum use. It benefits from the same optimization scheme to remove garbage bits, reducing overhead.

2. **Phase Circuits**

    These circuits are intrinsically quantum and the paper call these as "phase circuits".
    One is optimized for qubit count and the other for gate count and depth.

    The general idea of these phase circuits is to leverages the logic of the quantum phase estimation (QPE) algorithm to compute  $x^2 \ \textrm{mod} \ N$ as phase and "estimate" the phase, i.e. to encode $x^2 \ \textrm{mod} \ N$ into the state, that is desired result.

    These circuits use controlled phase rotations directly to compute $x^2 \ \textrm{mod} \ N $ in a decomposition of phases into the relative phases of the computational basis states in superposition (previously prepared with a Hadamard transform).
    It then employs the inverse of the quantum Fourier transform to combine the relative phases into one value (the estimated phase) encoded in the computational basis.

* **Qubit optimized phase circuit.** It does not use the whole inverse quantum Fourier transform, instead it exploits measurement of one qubit of the output register immediately after is computed by with the controlled phase rotation. This reduces the number qubits used for the output register. But it requires to perform mid-circuit measurements and re-using the same output qubit, or run multiple times the circuit to compute the different bits of the final output.

* **Depth optimized phase circuit.** It actually uses the whole inverse quantum Fourier transform as described above. And to optimize for gates it leverages an addditional ancilla quantum register called "counter", to counts when for certains pairs of indices of qubits in the controlled phase rotations are both 1. This reduces the gate counts.


**Experimental implementation circuit**

The paper propose also a version of the phase estimation circuits, specific for quantum hardware of neutral atoms coupled with Ridberg states. It further reduces gate count and depth by leveraging the Rydberg blockade mechanism, which natively supports multi-qubit controlled phase gates, thereby minimizing the computational complexity.

* **Paper's code**

The supplementary materials of the paper include a [public reposistory](https://github.com/GregDMeyer?tab=repositories) with the source code for the four quantum circuits illstrated above. The code is written with the Python SDK Cirq.

* **Our implemantaion and following sections**

Among the four circuits provided by the paper we choosed to implement the cirucit that requires less qubits, in order to test it on the Classiq's simulator, which max number of qubits supported is 25. The circuits that requires less qubits are the Phase Circuits.

And among the 2 proposed phase circuits, the qubit optimized circuit (that in the code is called "narrow" circuit) is the cheapest in terms of qubit usage.
However, in order to implement it with also the quantum circuits that implement the other quantum steps of the protcol, it requires the use of mid-circuit measurements, and this is not supported by Classiq's platform.

For this reason _we chosed to implement the gate-optimized phase circuit_ , that in the code is called "fast" circuit, althogh it requries a higher number of qubits, due to the use of additional ancilla qubits.

In the following sections and cells, we will provide the implementation of the functions from the original paper's code using Classiq Python SDK, that is the Qmod language implementation in Python. Qmod is the language developed by Classiq, available in two versions, one is the native Qmod and one Python SDK, that is the embedding of Qmod in Python. In the code cells we will insert the original authors code (commented) and below our implemantation in Classiq Python SDK.
The names of the sections follow the paper code function names.


(Text by:  Filippo Tramonto, Nahid Binandeh Dehaghani)

##### Library Imports and Package Installation

In [1]:
## TIP : run this cell 2 times, when you run this notebook on Colab or other enviroment where classiq pacckage is not installed
##       the second time removes the annoying installation log prints
try:
    from classiq import *
except ImportError:
    print("installing classiq...")
    !pip install --quiet classiq
    from classiq import *
    print("installed classiq.")

import numpy as np
import random

## CZpow() function
(Function developed by Filippo Tramonto, Ban Tran)

In Qmod, the gates most similar to three gates `ZPowGate`, `CZPowGate`, `CCZPowGate` of Cirq are `PHASE` for `ZPowGate` and `CPHASE` for `CZPowGate`. The main differences is a factor $\pi$ in the phase argument of the gates (we have to multiply the phase by $\pi$ to reproduce the gates of Cirq).

We have eliminated the option with `nqubits == 1` with `ZPowGate` (of the authors code), because `ZPowGate` is never used in the code. Therefore, there are two functions to translate in Qmod/Phython-SDK: `CZPowGate`, `CCZPowGate`.

In Qmod, there is no gate similar to `CCZPowGate`. To implement it, we use the Qmod `control()` function to double control the `PHASE` gate. For the `CZPowGate` gate, there is the `CPHASE` gate, which is equivalent to using `control()` to apply the `PHASE` gate controlled by one qubit.

Thus, the implementation of `CZPowGate`, `CCZPowGate` would be identical. Since in Qmod we can use the implicit casting from QBit to QArray (i.e. passing a QBit variable to a function where the argument is declared as QArray), we define a unique function both for `CZPowGate` and `CCZPowGate`, we call this function MCZPhase (Multi-Controlled-ZPhase gate).


(Text by: Filippo Tramonto)

In [2]:
## Original authors function in Cirq
####################################################
#def CZpow(phase, *qubits):
#    nqubits = len(qubits)

#    # we put most common case first
#    if nqubits == 3:
#        base_gate = cirq.CCZPowGate(exponent=phase)
#    elif nqubits == 2:
#        base_gate = cirq.CZPowGate(exponent=phase)
#    elif nqubits == 1:
#        base_gate = cirq.ZPowGate(exponent=phase)
#    else:
#        raise ValueError('too many control qubits')
#
#    return base_gate(*qubits)
#
############## Developed by #########################
# Filippo Tramonto, Ban Tran
###################################################
from classiq.qmod.symbolic import pi
@qfunc
def MCZPhase(phase: CReal, target: QBit, controls: QArray[QBit]):
    control(controls, lambda: PHASE(phase * pi , target))

## phase_add() function
(Function developed by Filippo Tramonto)

The original function in Cirq is designed to add phases to a series of qubits using controlled Z rotations. The function parameters are the phase to be added, the target qubits, and the control qubits. There is a loop that iterates over each qubit in the target array, calculates the phase for each qubit, and
yields a controlled-Z rotation (CZpow) for each target qubit with the calculated phase and given control qubits.

Our Classiq Python SDK code aims to achieve the same functionality but adapted to Qmod/Python SDK syntax and capabilities. The code is structured into three functions:
*   `phase_add_2qubits`: Handles the case where there are two control qubits. It binds two control qubits to an array and iterates over the target qubits, applying the phase_add_iteration function.
*   `phase_add_1qubit`: Handles the case with a single control qubit. It  iterates over the target qubits, applying the phase_add_iteration function with a single control qubit.
*   `phase_add_iteration`: The core iteration logic to compute the phase and apply the controlled phase gate. It calculates the phase for each target qubit and applies the MCZPhase function (a multi-controlled phase gate), which is equivalent to the author's CZpow.

We use the parameters `in_phase` for the phase to be added, `target` for the target qubits array, `control1`, `control2`, control for control qubits,
`k` for the current iteration index, and `controls` for the array of control qubits (in the case of multiple control qubits).

Key Differences:
Qmod uses explicit function bindings and iteration constructs tailored to its syntax. The phase calculation and control application logic remain consistent with the original author's code. MCZPhase is a generalized function to handle both single and double control qubits.

(Text by: Nahid Binandeh Dehaghani)


In [3]:
## Original function in Cirq
####################################################
#def phase_add(in_phase, target, controls):
#    for k, yk in enumerate(target):
#        phase = 2**(len(target) - k - 1) * in_phase % 2
#        yield CZpow(phase, yk, *controls)

############## Developed by #########################
# Filippo Tramonto
###################################################
# The converted function in Qmod/Classiq Python SDK
#################################################
@qfunc
def phase_add_2qubits(in_phase: CReal, target: QArray[QBit], control1: QBit, control2: QBit) -> None:
    controls = QArray('controls')
    bind([control1, control2],controls)
    repeat(count = target.len,
           iteration = lambda k: phase_add_iteration(k,in_phase,target,controls))
    bind(controls,[control1, control2])

@qfunc
def phase_add_1qubit(in_phase: CReal, target: QArray[QBit], control: QBit) -> None:
    repeat(count = target.len,
           iteration = lambda k: phase_add_iteration(k,in_phase,target,control))

@qfunc
def phase_add_iteration(k: CInt, in_phase: CReal, target: QArray[QBit], controls: QArray[QBit]) -> None:
    phase = 2**(target.len -k -1)* in_phase % 2
    MCZPhase(phase, target[k], controls)   # target[k] is yk-th qubit of target in orgiginal code

### Test of the phase_add function implemenation

The test checks if the model creation and program synthesis works, without errors.

In [4]:
## TEST OF phase_add_2qubits and phase_add_1qubit
############## Developed by #########################
# Filippo Tramonto, Ban Tran
###################################################
import math
@qfunc
def main(x: Output[QArray[QBit]], counter: Output[QArray[QBit]]):

    N = 21
    domain_max = (N+1)//2 - 1
    n_x = domain_max.bit_length()
    allocate(n_x,x)

    n_counter = int(math.log(n_x) + 1)
    allocate(n_counter, counter)

    in_phase= math.pi/3
    #phase_add_2qubits(in_phase, counter, x[0], x[2])
    phase_add_1qubit(in_phase, counter, x[2])

qmod = create_model(main)

# to run on Colab (the exception is excecuted only the first time you run in the current Colab session)
try:
    qprog = synthesize(qmod)
#except ClassiqAPIError:     # this does't work, it does not recognize the name "ClassiqAPIError"
except Exception:            # this captures every error...
    authenticate(overwrite=True)
    qprog = synthesize(qmod)

show(qprog)

Opening: https://platform.classiq.io/circuit/83013acc-5358-4eb0-910f-a16ee4cffb30?version=0.44.0


## count() function
(Function developed by Ban Tran, Filippo Tramonto)

The original author's function processes pairs of qubit indices, computes phases, and yields the results using the `phase_add` function. The parameters used are the quantum registers `x` and `counter`, `pairs`, a list of pairs of indices, and `sign`, a factor +1 or -1 used in phase calculation.
The function loops through each pair `(i, j)` in `pairs`.
If `i` is equal to `j`, a single control qubit,`x[i]`, is used, , and `fctr` is set to 1.
If `i` is not equal to `j`, two control qubits, `x[i]` and `x[j]` are used, and `fctr` is set to 2.
The phase `phase` is calculated using `sign`, `fctr`, and the length of `counter`. The `phase_add` function is called with the calculated `phase`, `counter`, and `controls`.

The converted function in Qmod/Classiq Python SDK performs the same operations but is adapted to Classiq's syntax and functionality.
The main difference in the implemantation in Classiq Python SDK is we had to split the phase_add function into two functions. The reason is explained below in the subsection on the challeges faced.

The `phase_add_1qubit` function is called if there is a single control qubit.

The `phase_add_2qubits` function is called if there are two control qubits.

**Challenges faced while implementing count() in Qmod/Classiq Python SDK**


1.   Inability to use classical Python expressions and the Classiq's `repeat()` function in substitution of the `for` loop.

* **Issue**: classical python expressions used in author's code could not be written in symbolic expressions with Classiq's variable types.  And we had similar issues in using the `repeat()` function.

* **Solution**: We removed `@qfunc` decorator to use classical python expressions and substituted the repeat function with the usual Python `for` loop to iterate over the pairs of qubit indices.

2.   Passing multiple qubits of a QArray as a single argument to a function:

* **Issue**: In Qmod/Classiq-Python-SDK, it is not possible to pass two qubits of a QArray register (`x[i]` and `x[ij]`) as a single QArray argument to the `phase_add` function, like it is done in Cirq author's code. Furthermore, a possible soulution came out to us was using the `bind` function to cast the two qubits into a single QArray varaible and then pass this to the function. This solution it is not possible, because passing two subscripted elements of QArray to `bind` is not supported.

* **Solution**: We addressed this by splitting the phase_add function into two separate functions, `phase_add_1qubit` and `phase_add_2qubits`, in order to handle separately the two cases of one control qubit and two control qubits. Additionally, we utilized the bind function within the `phase_add_2qubits` function to cast the control qubits of `x` register into one QArray variable to pass to the `MCZPhase` gate/function.

* Further Action: We reached out to Classiq on Slack to discuss and seek a resolution for this issue, and thanks to their replies we found the solution.

(Text by: Nahid Binandeh Dehaghani, Filippo Tramonto, Ban Tran)


In [5]:
## The original cirq function
#######################################################
# def count(x, counter, pairs, sign):
#     for i,j in pairs:
#         if i == j:
#             controls = [x[i]]
#             fctr = 1
#         else:
#             controls = [x[i], x[j]]
#             fctr = 2

#         phase = sign*fctr*(2**(1-len(counter)))
#         yield phase_add(phase, counter, controls)
######################################################
# The converted function in Qmod/Classiq Python SDK
#### Developed by ######################################
# Ban Tran, Filippo Tramonto
######################################################
def count(x: QArray[QBit], counter: QArray[QBit], pairs, sign) -> None:
    # Loop the array of qubit index pairs to set 1 or 2 qubits of x register as controls register. For example, the array could be [(1,2), (1, 3), (0,2)]
    for i,j in pairs:
        if i == j:
            # If two indexes are equal, set x[i] qubit as controls qubit
            fctr = 1
            # Calculate phase for the phase_add_1qubit function
            phase = sign *fctr *(2**(1-counter.len))
            # Call the phase_add_1qubit function
            phase_add_1qubit(phase, counter, x[i])
        else:
            # If two indexes are different, set 2 qubits x[i] and x[j] as the controls
            fctr = 2
            # Calculate phase for the phase_add_2qubits function
            phase = sign *fctr *(2**(1-counter.len))
            # Call the phase_add_2qubits function
            phase_add_2qubits(phase, counter, x[i], x[j])

### Test of the count function implemenation

The test checks if the model creation and program synthesis works, without errors.

In [6]:
# TEST CODE FOR MODEL CREATION AND SYNTHESIS OF count FUNCTION ####
############## Developed by #######################################
# Filippo Tramonto
###################################################################
import math
@qfunc
def main(x: Output[QArray[QBit]], counter: Output[QArray[QBit]]):

    N = 21
    domain_max = (N+1)//2 - 1
    n_x = domain_max.bit_length()
    allocate(n_x,x)

    n_counter = int(math.log(n_x) + 1)
    allocate(n_counter, counter)

    pairs = [(1,1),(0,2)]
    count(x, counter, pairs, sign=+1)

qmod = create_model(main)

# to run on Colab (the exception is excecuted only the first time you run in the current Colab session)
try:
    qprog = synthesize(qmod)
#except ClassiqAPIError:     # this does't work, it does not recognize the name "ClassiqAPIError"
except Exception:            # this captures every error...
    authenticate(overwrite=True)
    qprog = synthesize(qmod)

show(qprog)

Opening: https://platform.classiq.io/circuit/4f9b16ab-db71-43ea-8897-65d609ef73fc?version=0.44.0


## qft() and iqft() functions
(Function developed by Filippo Tramonto)

The original paper functions `qft` and `iqft`, coded in Cirq, implement the Quantum Fourier Transform (QFT) and its inverse (IQFT). However, when we attempted to use Classiq/Qmod's `qft` function (with `iqft` as its inverse obtained using the `invert()` function), the results did not match those produced by running the original paper's code. Specifically, during our comparison of the results from our Classiq implementation with those obtaind with the original code, we discovered the following:

When we used Classiq's `qft` and `iqft` functions, the results did not align, due to little differences in the actual implementations.

Conversely, when we translated the original qft function from the paper into Classiq, the results were identical to those of the original code.

In the following, we provide a detailed explanation of `qft` and `iqft` functions for the original code developed in Cirq and our implemented code developed in Classiq Python SDK.

The `qft` function in Cirq uses the parameter reg as an array of qubits. Then, iterates over the qubits in reverse order in an outer loop, and applies controlled phase rotations with a phase factor $1/2^{m-k}$ (in Qmod it would be $\pi/2^{m-k}$ due to the $\pi$ factor difference) in an inner loop. Then, applies a Hadamard gate to each qubit. For the `iqft` function, the phase factor is $-1/2^{m-k}$.

We called the functions translated in Classiq Python SDK, as `qft_custom` and `iqft_qustom`, to differentiate from the Classiq's `qft`.

The the translated function of the paper code `qft` fucntion is structured in 3 functions:

* `qft_custom`: The main function. The outer loop of the QFT is implemented with the `repeat` function that iterates over the qubits of the register, and the lambda function calls `qft_outer_iteration` for each index `k`.

* `qft_outer_iteration`: It handles the iteration of the outer loop. Here a index `kk` is calculated as `reg.len -k -1` to iterate in reverse order over the register, because with the `repeat` function the iterator index always goes from 0 to the total number of iterations (`count` parameter of `repeat`). For the inner loop, the `repeat` function iterates over the subsequent qubits starting from `kk + 1`.
The `count` parameter of the repeat funciton is defined as the total number of iterations of the inner loop, `reg.len -kk -1`, that in this case depends by the value of `kk`, because the `m` index goes from `kk +1` to the end of the register.
The lambda function calls `qft_inner_iteration` for each pair of indices `m` and `kk`. Then, after the innner loop, and inside the outer loop, a Hadamard gate (H) is applied to the qubit at index `kk`.

* `qft_inner_iteration`: QFT inner iteration function handles the iteration of the inner loop of the QFT. The index `mm` is  defined as `m + kk +1`  to get the correct original index starting from `kk + 1`. The phase calculation computes the phase shift as `1 / 2^(mm - kk)`. Then, the multi-controlled phase gate `MCZPhase` is applied to apply the controlled phase gate between the qubits at indices `mm` and `kk`.

For the IQFT, in our code in Classiq Python SDK the `iqft_custom` is implemented using the `invert` function to apply the inverse of the `qft_custom` function.

(Text: by Nahid Binandeh Dehaghani, Filippo Tramonto)

In [7]:
# The original cirq functions
###################################################
#def qft(reg):
#    for k, yk in list(enumerate(reg))[::-1]:
#        for m, ym in list(enumerate(reg))[k+1:]:
#            phase = 1/2**(m-k)
#            yield CZpow(phase, ym, yk)
#        yield cirq.H(yk)
#def iqft(reg):
#    for k, yk in enumerate(reg):
#        yield cirq.H(yk)
#        for m, ym in list(enumerate(reg))[k+1:]:
#            phase = -(1/2**(m-k))
#            yield CZpow(phase, ym, yk)
############## Developed by #########################
# Filippo Tramonto
###################################################
@qfunc
def qft_custom(reg: QArray[QBit]):
    # outer loop with index k
    repeat(count=reg.len,
          iteration= lambda k: qft_outer_iteration(k,reg) )

@qfunc
def qft_outer_iteration(k: CInt, reg: QArray[QBit]):
    kk = reg.len -k -1  # to have the loop in reverse order (from reg.len-1 to 0)
    # inner loop with index m
    repeat(count=reg.len-kk-1,    # in the orginal function the total number of iterations is len(reg)-k-1
          iteration= lambda m: qft_inner_iteration(m, kk, reg) )
    H(reg[kk])

@qfunc
def qft_inner_iteration(m: CInt, kk: CInt, reg: QArray[QBit]):
    mm = m + kk +1    # to have the original index m that starts from k+1
    phase = 1/2**(mm-kk)
    MCZPhase(phase, reg[mm], reg[kk])

@qfunc
def iqft_custom(reg: QArray[QBit]):
    invert(lambda: qft_custom(reg))

## x2modN_fast function
(Function developed by Filippo Tramonto)

The `x2modN_fast` function implements a part of an efficient quantum algorithm to calculate $𝑥^2\mod N$, that in the paper is called gate-optimezed "phase circuit". This circuit is one of the two versions of the phase circuit described in the paper.

The function leverages the quantum Fourier transform and controlled phase rotations to achieve this computation. Below is a detailed explanation of the function, both in its original Cirq form and its converted version in Classiq Python SDK.

We first delve into the explanation of the original function of the paper code in Cirq.

The number of qubits in the input register x is determined by `n = len(x)`. The instruction `counter = ancillas.new_register(n.bit_length())` allocates a new ancillary register `counter` with enough qubits to represent the bit length of `n`. This register will help track the phases needed for the computation. Then, the Hadamard Transform is applied the counter register, creating a superposition of all possible states. This step is important for setting up the quantum state for the subsequent phase estimation.

The computational loop iterates over `l` from `0` to `2n-1`, which represents different stages of the modular multiplication process. `qubit_pairs` is computed as pairs indices of qubits that will act as control qubits for the phase rotations. Then, the phases in the `counter` register are updated based on the input register `x` and the current pair of qubits and IQFT is applied to the `counter` register to convert the phase information into the computational basis.

For each qubit in the counter, a phase proportional to $2 \cdot 2^{k+l} / N$ is added to the `y` register, which accumulates the result of the modular multiplication.

To ensure the circuit is reversible, the inverse of the functions `qft` and `count` and the Hadamard Transfor are applied to uncompute the state, and thus restore the ancillary qubits (`counter`) to their initial states. The ancillary counter register is discarded after its role in the computation is complete.

The conversion to Classiq retains the structure and logic of the original Cirq implementation while adapting to the syntax and capabilities of the Classiq framework.

The adaptions that are made here to meet the Classiq syntax and capabilities are:

1. when the `counter` register is defined it is simply declared and initialized with the `allocate` function, instead in the original paper code it is used an `ancillas` class to manage the discard and reuse of the ancilla qubits.

2. we moved the initialization of `n_x` and `n_counter` the lenght of `x` register (in the original code is `x`) in the global scope.

3. we leveraged the `hadamard_trasnform` function provided by Classiq instead of itraating over the register qubits and applying the hadamard gate to each qubit

**Challenges faced while implementing x2modN_fast() in Classiq Python SDK**

In the translation of this function to Classiq framework, we faced  challenges similar to those met with the `count()` function.

* **Issues**: classical Python expressions used in author's code could not be written in symbolic expressions with Classiq's variable types, We had similar issues also in using the `repeat` function.

* **Solution**: We removed `@qfunc` decorator to use classical Python expressions and substituted the `repeat` function with the usual Python `for` loop to iterate over the pairs of qubit indices.

(Text by Nahid Binandeh Dehaghani, Filippo Tramonto)



In [8]:
## The original cirq function
#################################################################################
#def x2modN_fast(x, y, ancillas, N):
#    n = len(x)
#    counter = ancillas.new_register(n.bit_length())
#    for q in counter:
#        yield cirq.H(q)

#    for l in range(2*n-1):
#        qubit_pairs = [(i, l-i) for i in range(max(0, l-n+1), l//2 + 1)]

#        yield count(x, counter, qubit_pairs, sign=+1)
#        yield iqft(counter)

#        for k, ck in enumerate(counter):
#            phase = 2*2**(k+l) / N
#            yield phase_add(phase, y, [ck])

        # uncompute
#        yield qft(counter)
#        yield count(x, counter, qubit_pairs, sign=-1)

#    for q in counter:
#        yield cirq.H(q)

#    ancillas.discard(counter)
############## Developed by #########################
# Filippo Tramonto
###################################################
# The converted function in Qmod/Classiq Python SDK
##################################################
def x2modN_fast(x: QArray[QBit], y: QArray[QBit], N: int) -> None:
    counter = QArray("counter")
    allocate(n_counter, counter)
    hadamard_transform(counter)

    for l in range(2*n_x -1):
        qubit_pairs = [(i, l-i) for i in range(max(0, l-n_x+1), l//2 + 1)]

        count(x, counter, qubit_pairs, sign= 1)
        iqft_custom(counter)

        for k in range(n_counter):
            phase = 2 * 2**(k+l) / N
            phase_add_1qubit(phase, y, counter[k])

        # uncompute
        qft_custom(counter)
        count(x, counter, qubit_pairs, sign= -1)

    hadamard_transform(counter)

## Notes about the use of x2modN_fast function and other functions

When the function `x2modN_fast` is used in a test function like that defined in the code cell below, or the function `x2_mod_N_phase` described in the nex section, the classical parameters, `n_x`, `n_y`, `n_counter`  for the lenghts of the `x`, `y` and the ancilla `counter` registers have to be defined in the global scope.

`n_x` is computed as bit lenght of `domain_max` that defines the max value of the domain of $x^2 \mod N$, starting from 0 (i.e. the domain of $x^2 \mod N$ is [0,`domain_max`]). `domain_max` is defined as the last integer before $N/2$ (i.e. if $N$ is odd is the integer part of $N/2$, if $N$ even is $N/2-1$).

`n_y` is computed as bit lenght of $N$ (since the maxmium value for $y$ is $N-1$) plus the number of extra bits `n_extra_bits` required to increase the accuracy of the computation of  $x^2 \mod N$. This is due the approximated nature of the computation computation, limited by the number of bits/qubits to represent the binary fraction $w = x^2/N = 0.w_{m-1} w_{m-2} ... w_1 w_0$,  where $m$ is the total number of bits `n_y` and the bit strings $w_0$, $w_1$, ..., $w_{m-2}$, $w_{m-1}$ is the binary representation of the integer number that divided by $2^m$ gives the fraction $w = x^2/N $.

`n_counter` is computed as  the bit lenght of `n_x`.

(Text by Nahid Binandeh Dehaghani, Filippo Tramonto)

### Test of the x2modN_fast function implemenation

The test checks if the model creation and program synthesis works, without errors.

In [9]:
# TEST CODE FOR MODEL CREATION AND SYNTHESIS OF x2modN_fast FUNCTION
############## Developed by #########################
# Filippo Tramonto
###################################################
N = 21
n_extrabits = 0
# the next 2 lines, are taken from the authors's code in file check_phase_circuits.py (in main() function)
# domain_max is the max value of the domain of x^2 mod N, i.e. the max value for x (starting from 0)
# the domain of x^2 mod N, is [0,domain_max]
# the formula below means: if N is even, domain_max= N/2 -1 ; if N is odd domain_max= floor(N/2)
domain_max = (N+1)//2 - 1
n_x = domain_max.bit_length()
n_y = N.bit_length() + n_extrabits # in the orginal code: args.extra_bits
n_counter = n_x.bit_length()       # number of qubits of the ancilla register counter

@qfunc
def main(x: Output[QArray[QBit]], y: Output[QArray[QBit]]):
    allocate(n_x,x)
    allocate(n_y,y)   # in the orginal code: N.bit_length()+args.extra_bits

    x2modN_fast(x, y, N)

qmod = create_model(main)
write_qmod(qmod, "x2modN_fast-N=21")

# to run on Colab (the exception is excecuted only the first time you run in the current Colab session)
try:
    qprog = synthesize(qmod)
#except ClassiqAPIError:     # this does't work, it does not recognize the name "ClassiqAPIError"
except Exception:            # this captures every error...
    authenticate(overwrite=True)
    qprog = synthesize(qmod)

show(qprog)

Opening: https://platform.classiq.io/circuit/df051b1f-d6fb-40e3-8197-620a6f4994c2?version=0.44.0


## x2_mod_N_phase() function

(Function developed by Nahid Binandeh Dehaghani and Filippo Tramonto)

This function, `x2_mod_N_phase`, is designed to generate a quantum circuit for calculating $x^2 \mod N$.

The original code in Cirq provides the implemenations of 4 quantum circuits for the computation of $x^2 \mod N$, including the 2 "phase cirucits",  as they are called in the paper. In paper text they are called gate-optimized and qubit-optimized. These two cirucits in the Cirq code are called "fast" and "narrow" respectively.

Our Classiq implementation focuses only on the "fast" circuit.

Now, let's describe the original code function.
`x` indicates a quantum array (register) of qubits representing the values x of the Rabin's function. `y` is the quantum array of qubits representing the result register `y`, where the result of $x^2 \mod N$ will be stored, `ancillas`  is the an instance of the `ancilla` class defined in another part of the paper code, `circuit_type` is a string variable to choose the circuit among the "fast" and "narrow" circuit. $N$ is a classical integer representing the modulus $N$. The function starts by applying the Hadamard gate to every qubit in the x and y registers. The Hadamard gate (H) creates a superposition of all possible computatinal basis states, which is crucial for parallel quantum processing in many algorithms, including modular exponentiation.
This prepares the qubits for the subsequent quantum operations by placing them in an equal superposition of their basis states. The function `x2modN_fast` is called, which is responsible for performing the core computation of $x^2 \mod N$

Finally, the function applies the inverse quantum Fourier transform (iqft) to the y register to finalize the computation by ensuring that the result stored in the y register is in the correct form.

The translation to Classiq retains the structure and logic of the original Cirq implementation.

The main adaption made here is that we leveraged the `hadamard_trasnform` function provided by Classiq instead of itraating over the register qubits and applying the hadamard gate to each qubit

(Text by Nahid Binandeh Dehaghani, Filippo Tramonto)


In [10]:
#The original code in Cirq is copied below. However, we remember that we only do the fast part and we skip narrow.
#def x2_mod_N_phase(N, x, y, ancillas, circuit_type):
#    '''
#    Generate the full circuit
#    '''
#    for q in itertools.chain(x, y):
#        yield cirq.H(q)
#
#    if circuit_type == 'fast':
#        yield x2modN_fast(x, y, ancillas, N)
#        yield iqft(y)
#
#    elif circuit_type == 'narrow':
#        for k, yk in enumerate(y):
#            yield x2modN_narrow_single(x, [yk], ancillas, N, len(y)-k-1)
#            yield cirq.H(yk)
#            for m, ym in list(enumerate(y))[k+1:]:
#                phase = -(1/2**(m-k))
#                yield CZpow(phase, ym, yk)
#
#    else:
#        raise ValueError('unknown circuit type')
#######################################################

# We need to use x2modN_fast and iqft  defined in the previous cells

############## Developed by #########################
# Nahid Binandeh Dehaghani and Filippo Tramonto
############## Developed by #########################

# Generate the full circuit for the "fast" circuit type
@qfunc
def x2_mod_N_phase(x: QArray[QBit], y: QArray[QBit]) -> None:

    # Apply Hadamard gates to all qubits in x and y
    hadamard_transform(x)  # Apply Hadamard transform
    hadamard_transform(y)  # Apply Hadamard transform

    x2modN_fast(x, y, N)
    iqft_custom(y)

### Test of the x2_mod_N_phase function implemenation

The test checks if the model creation and program synthesis works, without errors.

In [11]:
# TEST CODE FOR MODEL CREATION AND SYNTHESIS OF x2_mod_N_phase FUNCTION
############## Developed by #########################
# Filippo Tramonto
###################################################
N = 21
n_extrabits = 0
# the next 2 lines, are taken from the authors's code in file check_phase_circuits.py (in main() function)
# domain_max is the max value of the domain of x^2 mod N, i.e. the max value for x (starting from 0)
# the domain of x^2 mod N, is [0,domain_max]
# the formula below means: if N is even, domain_max= N/2 -1 ; if N is odd domain_max= floor(N/2)
domain_max = (N+1)//2 - 1
n_x = domain_max.bit_length()
n_y = N.bit_length() + n_extrabits # in the orginal code: args.extra_bits
n_counter = n_x.bit_length()       # number of qubits of the ancilla register counter

@qfunc
def main(x: Output[QNum], y: Output[QNum]):
    allocate(n_x,x)
    allocate(n_y,y)   # in the orginal code: N.bit_length()+args.extra_bits

    x2_mod_N_phase(x, y)

qmod = create_model(main)
write_qmod(qmod, "main_x2_mod_N_phase-N=21")

# to run on Colab (the exception is excecuted only the first time you run in the current Colab session)
try:
    qprog = synthesize(qmod)
#except ClassiqAPIError:     # this does't work, it does not recognize the name "ClassiqAPIError"
except Exception:            # this captures every error...
    authenticate(overwrite=True)
    qprog = synthesize(qmod)

show(qprog)

Opening: https://platform.classiq.io/circuit/4275aeec-ce64-48ce-a787-540255352aa6?version=0.44.0


## Tests of phase circuit with our implementation and with paper code, wite the Statevecotor simulator

We have tested the whole phase circuit implemented in Classiq Python SDK , in order to compare directly with the results obtained by executing the author code. In the public repository part of the supplementary meteria of the paper there is a Python file in Circ SDK for checkin the correctness of the execution of the phase circuits and the accuracy of the computed values of $y = x^2 \!\!\! \mod \!\! N$ . This checking is performed by running the statevector simulator.

The tests (of the 2 codes) were performed with the same input paramters. And they gave indentical results.

### Test of phase circuit with OUR implementaion in Classiq Python SDK

#### Defintion the test function that performs all test, containing also the defintion of the quantum main() function

In [12]:
############## Developed and tested by #########################
# Filippo Tramonto, Ban Tran
#################################################################

## import and setting execution preferences to run on the statevector simulator
from classiq.execution import (
    ClassiqBackendPreferences,
    ClassiqSimulatorBackendNames,
    ExecutionPreferences,
)
from classiq.synthesis import set_execution_preferences
from classiq.execution import ExecutionDetails

backend_preferences = ClassiqBackendPreferences(
    backend_name=ClassiqSimulatorBackendNames.SIMULATOR_STATEVECTOR
)
execution_preferences=ExecutionPreferences(
    num_shots=1,
    backend_preferences=backend_preferences
)

## an utility function
def round_complex(x,digits):
    x_real = round(x.real,digits)
    x_imag = round(x.imag,digits)
    if x_real == 0:  # to force it to zero when after the round it's "-0.0"
        x_real = 0.0
    if x_imag == 0:
        x = round(x.real,digits)
    else:
        x = complex(x_real,x_imag)
    return x

## the actual function for testing whole phase circuit
def test_phase_circuit_classiq():
    print("N=",N)
    print("n_x:",n_x)
    print("n_y:",n_y)
    print("n_counter=",n_counter)
    print()

    # Defintion of the main quantum function
    @qfunc
    def main(x: Output[QNum], y: Output[QNum]):

        allocate(n_x,x)
        allocate(n_y,y)

        x2_mod_N_phase(x, y)

    ## creation of quantum model (it executes also the classical operations)
    qmod = create_model(main)

    ## setting execution preferences to run on the statevector simulator
    qmod = set_execution_preferences(
        qmod,
        execution_preferences=execution_preferences
    )

    write_qmod(qmod, f"x2_mod_N_phase-Statevector-N={N}")
    ## synthesize the quantum program
    qprog = synthesize(qmod)
    show(qprog)

    ## execution of the qunntum program
    job = execute(qprog)

    result = job.result()[0].value
    print("URL:",job.ide_url)
    state_vector = result.state_vector
    #for key, val in state_vector.items():
    #    val = round_complex(complex(val),3)
    #    print(f"{key}: {val}")

    print("=========================")
    parsed_state_vector = result.parsed_state_vector
    def myFunc(e):
        return e.bitstring
    parsed_state_vector.sort(key=myFunc)
    for state in parsed_state_vector:
        print(state.state,state.bitstring,round_complex(complex(state.amplitude),3))

#### Actual run of the test with different values of N

In [13]:
# input classical parameters
N = 5

# domain_max is the max value of the domain of x^2 mod N, i.e. the max value for x
# the domain of x^2 mod N, is [0,domain_max]
# the formula below means: if N is even, domain_max= N/2 -1 ; if N is odd domain_max= floor(N/2)
domain_max = (N+1)//2 - 1
n_x = domain_max.bit_length()
n_y = N.bit_length() + n_extrabits # in the orginal code: args.extra_bits
n_counter = n_x.bit_length()       # number of qubits of the ancilla register counter

#test execution
test_phase_circuit_classiq()

N= 5
n_x: 2
n_y: 3
n_counter= 2

Opening: https://platform.classiq.io/circuit/eede4cc9-de97-4de9-8765-49b0a602cbfe?version=0.44.0
URL: https://platform.classiq.io/jobs/a957fb95-2cf2-4b6a-882f-3038bc066b7d
{'x': 0.0, 'y': 0.0} 0000000 0.5
{'x': 1.0, 'y': 0.0} 0000001 (0.031+0.096j)
{'x': 2.0, 'y': 0.0} 0000010 (0.031-0.096j)
{'x': 3.0, 'y': 0.0} 0000011 (0.031-0.096j)
{'x': 0.0, 'y': 1.0} 0000100 0.0
{'x': 1.0, 'y': 1.0} 0000101 (-0.02+0.254j)
{'x': 2.0, 'y': 1.0} 0000110 (0.045-0.053j)
{'x': 3.0, 'y': 1.0} 0000111 (0.045-0.053j)
{'x': 0.0, 'y': 2.0} 0001000 0.0
{'x': 1.0, 'y': 2.0} 0001001 (0.173-0.339j)
{'x': 2.0, 'y': 2.0} 0001010 (0.054-0.027j)
{'x': 3.0, 'y': 2.0} 0001011 (0.054-0.027j)
{'x': 0.0, 'y': 3.0} 0001100 0.0
{'x': 1.0, 'y': 3.0} 0001101 (0.087-0.074j)
{'x': 2.0, 'y': 3.0} 0001110 (0.061-0.005j)
{'x': 3.0, 'y': 3.0} 0001111 (0.061-0.005j)
{'x': 0.0, 'y': 4.0} 0010000 -0.0
{'x': 1.0, 'y': 4.0} 0010001 (0.07-0.023j)
{'x': 2.0, 'y': 4.0} 0010010 (0.07+0.023j)
{'x': 3.0, 'y':

### Tests of author's code for the phase circuit for comparison

Here we also run the author's code for the phase circuit to compare with our results to check if we have implemented the functions in Qmod correctly.

#### Import and packace installation cell

In [14]:
## TIP : run this cell 2 times, when you run this notebook on Colab or other enviroment where cirq pacckage is not installed
##       the second time removes the annoying installation log prints
try:
    import cirq
except ImportError:
    print("installing classiq...")
    !pip install --quiet cirq==0.14.1
    print("installed cirq.")
    import cirq

import itertools

installing classiq...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.0/594.0 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.5/145.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

#### Authors code functions

**NOTE 1:** in order to run the authors functions, the two authors files ancillas.py and tof_sim.py are also neeeded. These files are in the same folder of this notebook. (If your run this notebook on Colab, you need to upload these files to Colab by clicking on the icon "upload to session storage" in the "Files" tab on the left).

NOTE 2: We have renamed some of the original paper functions, to avoid to overwrite our previous defined functions that have the same name of the orginal functions.

In [15]:
def x2_mod_N_phase_paper(N, x, y, ancillas, circuit_type):
    '''
    Generate the full circuit
    '''
    print(f"x len: {len(x)}")
    print(f"y len: {len(y)}")
    for q in itertools.chain(x, y):
        yield cirq.H(q)

    if circuit_type == 'fast':
        yield x2modN_fast_paper(x, y, ancillas, N)
        yield iqft(y)

    elif circuit_type == 'narrow':
        for k, yk in enumerate(y):
            yield x2modN_narrow_single(x, [yk], ancillas, N, len(y)-k-1)
            yield cirq.H(yk)
            for m, ym in list(enumerate(y))[k+1:]:
                phase = -(1/2**(m-k))
                yield CZpow(phase, ym, yk)

    else:
        raise ValueError('unknown circuit type')

def x2modN_narrow_single(x, y, ancillas, N, bit_idx):
    for i, xi in enumerate(x):
        for j, xj in list(enumerate(x))[i:]:
            if i == j:
                controls = [xi]
                fctr = 1
            else:
                controls = [xi, xj]
                fctr = 2

            phase = fctr*2*2**(i+j+bit_idx)/N
            yield phase_add(phase, y, controls)

def x2modN_fast_paper(x, y, ancillas, N):
    n = len(x)
    counter = ancillas.new_register(n.bit_length())
    print(f"counter len: {len(counter)}")
    for q in counter:
        yield cirq.H(q)

    for l in range(2*n-1):
        qubit_pairs = [(i, l-i) for i in range(max(0, l-n+1), l//2 + 1)]

        yield count_paper(x, counter, qubit_pairs, sign=+1)
        yield iqft(counter)

        for k, ck in enumerate(counter):
            phase = 2*2**(k+l) / N
            yield phase_add(phase, y, [ck])

        # uncompute
        yield qft(counter)
        yield count_paper(x, counter, qubit_pairs, sign=-1)

    for q in counter:
        yield cirq.H(q)

    ancillas.discard(counter)

def count_paper(x, counter, pairs, sign):
    for i,j in pairs:
        if i == j:
            controls = [x[i]]
            fctr = 1
        else:
            controls = [x[i], x[j]]
            fctr = 2

        phase = sign*fctr*(2**(1-len(counter)))
        yield phase_add(phase, counter, controls)

def phase_add(in_phase, target, controls):
    for k, yk in enumerate(target):
        phase = 2**(len(target) - k - 1) * in_phase % 2
        yield CZpow(phase, yk, *controls)

def iqft(reg):
    for k, yk in enumerate(reg):
        yield cirq.H(yk)
        for m, ym in list(enumerate(reg))[k+1:]:
            phase = -(1/2**(m-k))
            yield CZpow(phase, ym, yk)

def qft(reg):
    for k, yk in list(enumerate(reg))[::-1]:
        for m, ym in list(enumerate(reg))[k+1:]:
            phase = 1/2**(m-k)
            yield CZpow(phase, ym, yk)
        yield cirq.H(yk)

def CZpow(phase, *qubits):
    nqubits = len(qubits)

    # we put most common case first
    if nqubits == 3:
        base_gate = cirq.CCZPowGate(exponent=phase)
    elif nqubits == 2:
        base_gate = cirq.CZPowGate(exponent=phase)
    elif nqubits == 1:
        base_gate = cirq.ZPowGate(exponent=phase)
    else:
        raise ValueError('too many control qubits')

    return base_gate(*qubits)

#### Defintion the test function that performs all test


In [17]:
############## Developed and tested by ############
# Filippo Tramonto, Ban Tran
###################################################
from ancilla import AncillaManager
import numpy as np

def test_phase_circuit_paper():

    print("N=",N)
    # domain_max is the max value of the domain of x^2 mod N, i.e. the max value for x
    # the domain of x^2 mod N, is [0,domain_max]
    # the formula below means: if N is even, domain_max= N/2 -1 ; if N is odd domain_max= floor(N/2)
    domain_max = (N+1)//2 - 1
    n_x = domain_max.bit_length()
    n_y = N.bit_length() # in the orginal code: N.bit_length()+args.extra_bits
    print("n_x:",n_x)
    print("n_y:",n_y)
    print()

    # instantiating the quantum registers
    x = cirq.NamedQubit.range(n_x, prefix='x')
    y = cirq.NamedQubit.range(n_y, prefix='y')

    # instantiating the circuit
    circuit = cirq.Circuit()

    circuit_type='fast'
    ancillas = AncillaManager()
    circuit.append(x2_mod_N_phase_paper(N,x,y,ancillas,circuit_type))

    # instantiating the simluator
    simulator = cirq.Simulator()

    anc_reg = ancillas.all()
    #collecting the results
    #result = simulator.simulate(circuit,qubit_order=[*x[::-1],*y[::-1]])
    result = simulator.simulate(circuit,qubit_order=[*y[::-1],*anc_reg[::-1],*x[::-1]])

    for ampl in result.final_state_vector:
        print(np.around(np.real_if_close(ampl,2), 3))

    #print(cirq.dirac_notation(result.final_state_vector))

#### Actual run of the test with different values of N

In [19]:
# input classical parameters
N = 5

#test execution
test_phase_circuit_paper()

N= 5
n_x: 2
n_y: 3

x len: 2
y len: 3
counter len: 2
0.5
(0.031+0.096j)
(0.031-0.096j)
(0.031-0.096j)
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
(-0.02+0.254j)
(0.045-0.053j)
(0.045-0.053j)
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
(0.173-0.339j)
(0.054-0.027j)
(0.054-0.027j)
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
(0.087-0.074j)
(0.061-0.005j)
(0.061-0.005j)
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
(0.07-0.023j)
(0.07+0.023j)
(0.07+0.023j)
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
(0.061+0.005j)
(0.087+0.074j)
(0.087+0.074j)
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
(0.054+0.027j)
(0.173+0.339j)
(0.173+0.339j)
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
0.0
(0.045+0.053j)
(-0.02-0.254j)
(-0.02-0.254j)
0.0
0.0
0.0
0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0
-0.0


## Test of whole phase circuit with statevector simulator and getting the y values and success probability

The Python file in Circ SDK of the paper code, moentioned in the previous section, that checks the correctness of the execution of the phase circuits and the accuracy of the computed values of $y = x^2 \!\!\! \mod \!\! N$ , produces as outcome a cuccess probability to obtain enough accurate values of y, i.e. if they are rounded to the integer they are correct y. In other words, the code computes the cumulative probabiities of the y values that after rounding to integer are equal to the classical computed $x^2 \!\!\! \mod \!\! N$ with the x value taken from the same state of the statevector.

In our code below in Classiq Python SDK the same checking is reproduced. Of course, the parsing of the results of the Statevecotr simulator is different. Here with the SDK was a little more simple.

Also here, the success probabilities obtained are identical to those obtained with the paper code. The paper Python file to run is check_phase_circuits.py. From command line, the execution command to run it is: `python check_phase_circuits.py 5 -t fast --extra-bits 0` , where 5 is N=5 , and 0 is corresponds to `n_extrabits = 0` for our code.





### Defintion of main function for test and of the test function that performs all test for success probability of y with OUR CODE

In [20]:
def test_phase_circuit_succ_prob():
    print("N=",N)
    print("n_x:",n_x)
    print("n_y:",n_y)
    print("n_counter:",n_counter)
    print()

    # Defintion of the main quantum function
    @qfunc
    def main(x: Output[QNum], y: Output[QNum]):

        allocate(n_x,x)
        allocate(n_y, y)

        x2_mod_N_phase(x, y)

    ## creation of quantum model (it executes also the classical operations)
    qmod = create_model(main)
    ## setting execution preferences to run on the statevector simulator
    qmod = set_execution_preferences(
        qmod,
        execution_preferences=execution_preferences
    )
    print("synthesis...")
    # to run on Colab (the exception is excecuted only the first time you run in the current Colab session)
    try:
        qprog = synthesize(qmod)
    #except ClassiqAPIError:     # this does't work, it does not recognize the name "ClassiqAPIError"
    except Exception:            # this captures every error...
        authenticate(overwrite=True)
        qprog = synthesize(qmod)

    print("executing...")
    job = execute(qprog)
    result = job.result()[0].value
    print("finished!")
    return result

### Test with different values of N

In [21]:
# input classical parameters
N = 21
n_extrabits = 0

# domain_max is the max value of the domain of x^2 mod N, i.e. the max value for x
# the domain of x^2 mod N, is [0,domain_max]
# the formula below means: if N is even, domain_max= N/2 -1 ; if N is odd domain_max= floor(N/2)
domain_max = (N+1)//2 - 1
n_x = domain_max.bit_length() + n_extrabits
n_y = N.bit_length() # in the orginal code: N.bit_length()+args.extra_bits
n_counter = n_x.bit_length()

#test execution
result=test_phase_circuit_succ_prob()

N= 21
n_x: 4
n_y: 5
n_counter: 3

synthesis...
executing...
finished!


In [22]:
parsed_state_vector = result.parsed_state_vector
def myFunc(e):
    return e.bitstring
parsed_state_vector.sort(key=myFunc)

n_good_y = 0
good_count = 0
x_y_prob= {}
for state in parsed_state_vector:
    #print(state.state,state.bitstring,round_complex(complex(state.amplitude),3))

    x = int(state.state['x'])
    w_int = int(state.state['y']) # it's the integer represenation of the bitstring of the binary fraction w
    w = w_int / 2**n_y  # the binary fraction w
    y = w * N

    ampl=state.amplitude
    prob = (ampl*np.conj(ampl)).real

    key = (y, x)
    if key not in x_y_prob:
        x_y_prob[key] = 0
    x_y_prob[key] += prob

good_prob = 0
for (y, x), prob in x_y_prob.items():

    y_from_x = x**2 % N

    if abs(round(y) - y_from_x) < 1:
        n_good_y += 1
        good_prob += prob

print("success y probablility:",round(good_prob,3) )

success y probablility: 0.874
